In [1]:
import numpy as np

In [2]:
class RNN:
    def __init__(self, wx, wh, b):
        self.params = [wx, wh, b]
        self.grads = [np.zeros_like(wx), np.zeros_like(wh), np.zeros_like(b)]
        self.cache = None
    
    def forward(self, x, h_prev):
        wx, wh, b = self.params
        t = np.dot(h_prev, wh)+np.dot(x, wx)+b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next):
        wx, wh, b = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dwh = np.dot(h_prev.T, dt)
        dh_prev = np.dot(dt, wh.T)
        dwx = np.dot(x.T, dt)
        dx = np.dot(dt, wx.T)
        
        self.grads[0][...] = dwx
        self.grads[1][...] = dwh
        self.grads[2][...] = db
        
        return dx, dh_prev
    

In [7]:
class TimeRNN:
    def __init__(self, wx, wh, b, stateful=False):
        self.params = [wx, wh, b]
        self.grads = [np.zeros_like(wx), np.zeros_like(wh), np.zeros_like(b)]
        self.layers = None
        
        self.h, self.dh = None, None
        self.stateful = stateful
        
    def set_state(self, h):
        self.h = h
    
    def reset_state(self):
        self.h = None
        
    def forward(self, xs):
        wx, wh, b = self.params
        N, T, D = xs.shape
        D, H = wx.shape
        
        self.layers = []
        hs = np.empty((N, T, H), dtype='f')
        
        if not self.stateful or self.h is None:
            self. h = np.zeros((N, H), dtype='f')
        
        for t in range(T):
            layer = RNN(*self.params) # * はリストなどを分解して渡すことを意味する
            self.h = layer.forward(xs[:, t, :], self.h) # h_prevから時刻tのｈを作成
            hs[:, t, :] = self.h # 時刻tのhを格納
            self.layers.append(layer)
            
        return hs
    
    def backward(self, dhs):
        wx, wh, b = self.params
        N, T, H = dhs.shape
        D, H = wx.shape
        
        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh) # 合算した勾配
            dxs[:, t, :] = dx
            
            for i, grad in enumerate(layer.grads):
                self.grads[i] += grad
        
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh
        
        return dxs
    

In [24]:
import sys
sys.path.append('..')
from common.time_layers import *

class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        # 重みの初期化(Xavierの初期値)
        embed_w = (rn(V, D) / 100).astype('f')
        rnn_wx = (rn(D, H) / np.sqrt(D)).astype('f')
        rnn_wh = (rn(H, H) / np.sqrt(H)).astype('f')
        rnn_b = np.zeros(H).astype('f')
        affine_w = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        # レイヤの生成
        self.layers = [
            TimeEmbedding(embed_w),
            TimeRNN(rnn_wx, rnn_wh, rnn_b, stateful=True),
            TimeAffine(affine_w, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]
        
        # すべての重みと勾配をリストにまとめる
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, xs, ts):
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self):
        self.rnn_layer.reset_state()
    

In [25]:
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
from common.optimizer import SGD
from dataset import ptb
# from simple_rnnlm import SimpleRnnlm

# ハイパーパラメーターの設定
batch_size = 10
wordvec_size = 100
hidden_size = 100
time_size = 5 # Truncated BPTT の展開する時間サイズ
lr = 0.1
max_epoch = 100

#学習データの読み込み（データセットを小さくする）
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1]
ts = corpus[1:]
data_size = len(xs)
print('corpus size: {} , vocabulary size: {}'.format(corpus_size, vocab_size))

# 学習時に使用する変数
max_iters = data_size // (batch_size * time_size)
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

# モデルの生成
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)


corpus size: 1000 , vocabulary size: 418


In [33]:
# ミニバッチの各サンプルの読み込み開始位置を計算
jump = (corpus_size - 1) // batch_size
offsets = [i * jump for i in range(batch_size)]

In [36]:
print(jump)
print(offsets)
print(max_iters)

99
[0, 99, 198, 297, 396, 495, 594, 693, 792, 891]
19


In [40]:
for epoch in range(max_epoch):
    for iter in range(max_iters):
        # ミニバッチの取得
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
            time_idx += 1
        
        # 勾配を求め、パラメーターを更新
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1
    
    #エポック毎にパープレキシティの評価
    ppl = np.exp(total_loss / loss_count)
    print('| epoch {} | perplexity {}'.format(epoch+1, ppl))
    total_loss, loss_count = 0, 0
    

| epoch 1 | perplexity 319.9095499909084
| epoch 2 | perplexity 228.0625413561379
| epoch 3 | perplexity 214.70397216620427
| epoch 4 | perplexity 204.60185065157677
| epoch 5 | perplexity 199.6166219409499
| epoch 6 | perplexity 200.82618526058175
| epoch 7 | perplexity 196.58991923104267
| epoch 8 | perplexity 194.8412684659621
| epoch 9 | perplexity 189.49757977914103
| epoch 10 | perplexity 188.3140973342415
| epoch 11 | perplexity 192.55164664566792
| epoch 12 | perplexity 186.82416470488226
| epoch 13 | perplexity 190.71239448869727
| epoch 14 | perplexity 186.85053113644756
| epoch 15 | perplexity 185.91763971110217
| epoch 16 | perplexity 185.14550844938717
| epoch 17 | perplexity 187.19349825616453
| epoch 18 | perplexity 182.36828188246133
| epoch 19 | perplexity 178.39877713840505
| epoch 20 | perplexity 175.81297013296353
| epoch 21 | perplexity 174.76558524096808
| epoch 22 | perplexity 177.17941461370566
| epoch 23 | perplexity 177.17253351813707
| epoch 24 | perplexity 1